In [1]:
"""
Generate reviews.csv for Data Science Course - NLP Tasks
=========================================================
Uses Faker library for realistic customer reviews
Includes sentiment labels, aspect-based reviews, and text variations

Run: pip install faker pandas numpy textblob
Note: Requires customers.csv, products.csv, and transactions.csv
"""

import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import time
import random

# Initialize
fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)

print("="*70)
print("GENERATING REVIEWS.CSV FOR NLP TASKS")
print("="*70)

# Configuration
NUM_REVIEWS = 25000  # 25% of transactions get reviews
MIN_REVIEW_LENGTH = 10  # words
MAX_REVIEW_LENGTH = 150  # words

start_time = time.time()
print(f"\nGenerating {NUM_REVIEWS:,} customer reviews...")

# ============================================================================
# LOAD PREREQUISITE DATA
# ============================================================================

print("  → Loading existing datasets...")

try:
    customers_df = pd.read_csv('customers.csv')
    products_df = pd.read_csv('products.csv')
    transactions_df = pd.read_csv('transactions.csv')
    print(f"     ✓ Loaded {len(customers_df):,} customers")
    print(f"     ✓ Loaded {len(products_df):,} products")
    print(f"     ✓ Loaded {len(transactions_df):,} transactions")
except FileNotFoundError as e:
    print(f"\n❌ ERROR: {e}")
    print("Please run all previous generators first!")
    exit(1)

# ============================================================================
# REVIEW TEMPLATES BY SENTIMENT AND CATEGORY
# ============================================================================

print("  → Setting up review templates...")

# Positive review templates (sentiment: positive)
positive_templates = {
    'quality': [
        "Excellent quality product! {specific_praise}",
        "Outstanding quality! {specific_praise}",
        "Premium quality, {specific_praise}",
        "Top-notch quality! {specific_praise}",
        "Superb quality, highly recommended! {specific_praise}"
    ],
    'value': [
        "Great value for money! {specific_praise}",
        "Worth every rupee! {specific_praise}",
        "Excellent price point. {specific_praise}",
        "Best deal I've found! {specific_praise}",
        "Amazing value! {specific_praise}"
    ],
    'delivery': [
        "Fast delivery! {specific_praise}",
        "Received quickly, {specific_praise}",
        "Prompt delivery. {specific_praise}",
        "Delivered on time! {specific_praise}",
        "Super fast shipping! {specific_praise}"
    ],
    'overall': [
        "Absolutely love it! {specific_praise}",
        "Highly satisfied! {specific_praise}",
        "Exceeded expectations! {specific_praise}",
        "Perfect product! {specific_praise}",
        "Couldn't be happier! {specific_praise}"
    ]
}

# Negative review templates (sentiment: negative)
negative_templates = {
    'quality': [
        "Poor quality. {specific_complaint}",
        "Disappointed with quality. {specific_complaint}",
        "Not good quality. {specific_complaint}",
        "Quality issues! {specific_complaint}",
        "Substandard quality. {specific_complaint}"
    ],
    'value': [
        "Overpriced! {specific_complaint}",
        "Not worth the money. {specific_complaint}",
        "Too expensive for what you get. {specific_complaint}",
        "Poor value. {specific_complaint}",
        "Waste of money! {specific_complaint}"
    ],
    'delivery': [
        "Late delivery! {specific_complaint}",
        "Delivery was very slow. {specific_complaint}",
        "Took forever to arrive. {specific_complaint}",
        "Delayed shipping. {specific_complaint}",
        "Never delivered on time. {specific_complaint}"
    ],
    'overall': [
        "Very disappointed. {specific_complaint}",
        "Not satisfied. {specific_complaint}",
        "Regret buying this. {specific_complaint}",
        "Don't recommend. {specific_complaint}",
        "Won't buy again. {specific_complaint}"
    ]
}

# Neutral review templates (sentiment: neutral)
neutral_templates = [
    "It's okay. {neutral_comment}",
    "Average product. {neutral_comment}",
    "Nothing special. {neutral_comment}",
    "Does the job. {neutral_comment}",
    "Decent. {neutral_comment}",
    "It's fine. {neutral_comment}"
]

# Category-specific praise/complaints
category_specifics = {
    'Electronics': {
        'positive': [
            "Battery life is excellent.",
            "Performance is smooth and fast.",
            "Screen quality is amazing.",
            "Build quality feels premium.",
            "Features work perfectly.",
            "Sound quality is superb.",
            "Camera quality exceeded expectations.",
            "Processing speed is impressive."
        ],
        'negative': [
            "Battery drains too quickly.",
            "Performance is laggy.",
            "Screen has quality issues.",
            "Build feels cheap.",
            "Many features don't work.",
            "Sound quality is poor.",
            "Camera is disappointing.",
            "Too slow for the price."
        ],
        'neutral': [
            "Battery is average.",
            "Performance is okay for basic tasks.",
            "Screen is decent.",
            "Standard build quality."
        ]
    },
    'Clothing': {
        'positive': [
            "Fits perfectly!",
            "Fabric quality is great.",
            "Color is exactly as shown.",
            "Very comfortable to wear.",
            "Stitching is excellent.",
            "Material feels premium.",
            "Perfect size."
        ],
        'negative': [
            "Size doesn't fit properly.",
            "Fabric quality is poor.",
            "Color is different from image.",
            "Uncomfortable to wear.",
            "Stitching came loose.",
            "Material feels cheap.",
            "Wrong size received."
        ],
        'neutral': [
            "Fit is okay.",
            "Fabric is decent.",
            "Color is close to the picture."
        ]
    },
    'Home & Kitchen': {
        'positive': [
            "Very functional and practical.",
            "Sturdy construction.",
            "Easy to use and clean.",
            "Looks great in my home.",
            "Durable and long-lasting.",
            "Space-saving design."
        ],
        'negative': [
            "Broke within days.",
            "Poor construction quality.",
            "Difficult to clean.",
            "Doesn't look as good as picture.",
            "Not durable at all.",
            "Takes too much space."
        ],
        'neutral': [
            "Functional enough.",
            "Average build.",
            "Does what it's supposed to do."
        ]
    },
    'Books': {
        'positive': [
            "Engaging story!",
            "Well-written and informative.",
            "Couldn't put it down.",
            "Highly educational.",
            "Great read!",
            "Content is excellent."
        ],
        'negative': [
            "Boring content.",
            "Poor writing quality.",
            "Couldn't finish it.",
            "Not informative at all.",
            "Waste of time.",
            "Content is outdated."
        ],
        'neutral': [
            "Readable.",
            "Average content.",
            "It's okay for casual reading."
        ]
    },
    'default': {
        'positive': [
            "Works as expected.",
            "Good product overall.",
            "Serves the purpose well.",
            "Quality meets expectations.",
            "Happy with the purchase."
        ],
        'negative': [
            "Doesn't work as expected.",
            "Not satisfied overall.",
            "Doesn't serve the purpose.",
            "Quality below expectations.",
            "Unhappy with purchase."
        ],
        'neutral': [
            "It's acceptable.",
            "Average product.",
            "Nothing to complain about."
        ]
    }
}

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def get_category_specific(category, sentiment):
    """Get category-specific praise/complaint"""
    cat_key = category if category in category_specifics else 'default'
    specifics = category_specifics[cat_key]
    
    if sentiment == 'positive':
        return random.choice(specifics['positive'])
    elif sentiment == 'negative':
        return random.choice(specifics['negative'])
    else:
        return random.choice(specifics['neutral'])

def generate_review_text(rating, category, product_name, review_length='medium'):
    """Generate realistic review text based on rating"""
    
    # Determine sentiment from rating
    if rating >= 4.0:
        sentiment = 'positive'
    elif rating <= 2.5:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    # Start building review
    review_parts = []
    
    # Main statement
    if sentiment == 'positive':
        aspect = random.choice(list(positive_templates.keys()))
        template = random.choice(positive_templates[aspect])
        specific = get_category_specific(category, sentiment)
        main_statement = template.format(specific_praise=specific)
    elif sentiment == 'negative':
        aspect = random.choice(list(negative_templates.keys()))
        template = random.choice(negative_templates[aspect])
        specific = get_category_specific(category, sentiment)
        main_statement = template.format(specific_complaint=specific)
    else:
        template = random.choice(neutral_templates)
        specific = get_category_specific(category, sentiment)
        main_statement = template.format(neutral_comment=specific)
    
    review_parts.append(main_statement)
    
    # Add length variation
    if review_length in ['medium', 'long']:
        # Add another aspect
        if sentiment == 'positive':
            additional = random.choice([
                "Would definitely recommend to others.",
                "Will purchase again.",
                "Thank you for this amazing product!",
                "Five stars from me!",
                "Impressed with the seller too.",
                "Packaging was also excellent.",
                "Customer service was helpful."
            ])
        elif sentiment == 'negative':
            additional = random.choice([
                "Would not recommend.",
                "Won't buy again.",
                "Very disappointed with this purchase.",
                "Expected much better.",
                "Seller should improve quality.",
                "Packaging was also poor.",
                "Customer service was unhelpful."
            ])
        else:
            additional = random.choice([
                "It's acceptable for the price.",
                "Could be better.",
                "Average experience overall.",
                "Nothing to rave about."
            ])
        review_parts.append(additional)
    
    if review_length == 'long':
        # Add more details
        if sentiment == 'positive':
            extra = random.choice([
                f"I've been using this {category.lower()} product for a while now.",
                "My family also loves it.",
                "Great addition to my collection.",
                "Exactly what I was looking for!"
            ])
        elif sentiment == 'negative':
            extra = random.choice([
                "Had high expectations but let down.",
                "Multiple issues with this product.",
                "Should have read reviews first.",
                "Looking for a replacement now."
            ])
        else:
            extra = random.choice([
                "Serves basic needs.",
                "Not bad, not great either.",
                "You get what you pay for."
            ])
        review_parts.append(extra)
    
    # Add occasional product mention
    if random.random() < 0.3:  # 30% mention product
        review_parts.insert(0, f"Bought the {product_name.split()[0]} product.")
    
    # Join parts
    review_text = " ".join(review_parts)
    
    # Add some typos and casual language occasionally (realistic!)
    if random.random() < 0.15:  # 15% have casual language
        casual_replacements = {
            'excellent': 'excellent!!',
            'good': 'gud',
            'great': 'gr8',
            'product': 'product..',
            'quality': 'Quality',
            'disappointed': 'very disappointed',
            'love': 'loooove',
            'waste': 'total waste'
        }
        for word, replacement in casual_replacements.items():
            if word in review_text.lower() and random.random() < 0.5:
                review_text = review_text.replace(word, replacement)
    
    return review_text, sentiment

# ============================================================================
# SELECT TRANSACTIONS TO REVIEW
# ============================================================================

print("  → Selecting transactions for reviews...")

# Only completed transactions can have reviews
completed_transactions = transactions_df[
    transactions_df['Status'] == 'Completed'
].copy()

print(f"     ✓ {len(completed_transactions):,} completed transactions available")

# Sample transactions for reviews (higher rated products get more reviews)
if len(completed_transactions) > NUM_REVIEWS:
    # Merge with products to get ratings
    trans_with_products = completed_transactions.merge(
        products_df[['Product_ID', 'Rating']], 
        on='Product_ID'
    )
    
    # Weight by product rating (higher rated = more likely to be reviewed)
    weights = (trans_with_products['Rating'] ** 2).values
    weights = weights / weights.sum()
    
    selected_indices = np.random.choice(
        len(trans_with_products),
        size=NUM_REVIEWS,
        replace=False,
        p=weights
    )
    
    transactions_to_review = trans_with_products.iloc[selected_indices].copy()
else:
    transactions_to_review = completed_transactions.copy()

print(f"     ✓ Selected {len(transactions_to_review):,} transactions for reviews")

# ============================================================================
# GENERATE REVIEWS
# ============================================================================

print("  → Generating review texts...")

review_data = []
review_id = 1

for idx, transaction in transactions_to_review.iterrows():
    # Get related data
    product = products_df[
        products_df['Product_ID'] == transaction['Product_ID']
    ].iloc[0]
    
    customer = customers_df[
        customers_df['Customer_ID'] == transaction['Customer_ID']
    ].iloc[0]
    
    # Rating (with some variation from product rating)
    product_rating = product['Rating']
    # Customer rating varies slightly from product rating
    customer_rating = product_rating + np.random.normal(0, 0.8)
    customer_rating = round(np.clip(customer_rating, 1, 5), 1)
    
    # Review length (varied)
    length_choice = np.random.choice(
        ['short', 'medium', 'long'],
        p=[0.4, 0.4, 0.2]
    )
    
    # Generate review text
    review_text, sentiment = generate_review_text(
        customer_rating,
        product['Category'],
        product['Product_Name'],
        length_choice
    )
    
    # Review date (after transaction date)
    trans_date = pd.to_datetime(transaction['Transaction_Date'])
    review_delay_days = int(np.random.exponential(scale=7))  # Average 7 days
    review_delay_days = min(review_delay_days, 90)  # Max 90 days
    review_date = trans_date + timedelta(days=review_delay_days)
    
    # Helpful votes (positive reviews get more)
    if sentiment == 'positive':
        helpful_votes = int(np.random.exponential(scale=5))
    elif sentiment == 'negative':
        helpful_votes = int(np.random.exponential(scale=3))
    else:
        helpful_votes = int(np.random.exponential(scale=2))
    helpful_votes = min(helpful_votes, 500)
    
    # Verified purchase (95% are verified)
    verified_purchase = np.random.random() < 0.95
    
    # Review title (short summary)
    if sentiment == 'positive':
        titles = ['Great product!', 'Highly recommended', 'Love it!', 
                 'Excellent', 'Perfect', 'Amazing quality', 'Best purchase']
    elif sentiment == 'negative':
        titles = ['Disappointed', 'Not good', 'Poor quality', 
                 'Waste of money', 'Not recommended', 'Bad experience']
    else:
        titles = ['Okay', 'Average', 'Its fine', 'Decent', 'Could be better']
    
    review_title = random.choice(titles)
    
    # Reviewer name (anonymous or real)
    if np.random.random() < 0.3:  # 30% anonymous
        reviewer_name = 'Anonymous'
    else:
        reviewer_name = customer['First_Name']
    
    # Aspects (for aspect-based sentiment analysis)
    aspects_mentioned = []
    if 'quality' in review_text.lower():
        aspects_mentioned.append('quality')
    if 'price' in review_text.lower() or 'money' in review_text.lower():
        aspects_mentioned.append('price')
    if 'delivery' in review_text.lower() or 'shipping' in review_text.lower():
        aspects_mentioned.append('delivery')
    if 'service' in review_text.lower():
        aspects_mentioned.append('service')
    
    aspects_str = ', '.join(aspects_mentioned) if aspects_mentioned else 'general'
    
    review_data.append({
        'Review_ID': f'REV_{review_id:07d}',
        'Transaction_ID': transaction['Transaction_ID'],
        'Customer_ID': transaction['Customer_ID'],
        'Product_ID': transaction['Product_ID'],
        'Reviewer_Name': reviewer_name,
        'Rating': customer_rating,
        'Review_Title': review_title,
        'Review_Text': review_text,
        'Review_Date': review_date.date(),
        'Verified_Purchase': verified_purchase,
        'Helpful_Votes': helpful_votes,
        'Sentiment': sentiment,
        'Aspects_Mentioned': aspects_str,
        'Review_Length_Words': len(review_text.split())
    })
    
    review_id += 1
    
    # Progress indicator
    if review_id % 5000 == 0:
        progress = (review_id / NUM_REVIEWS) * 100
        print(f"     Progress: {progress:.1f}% ({review_id:,} / {NUM_REVIEWS:,})")

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

print("  → Assembling dataframe...")
reviews_df = pd.DataFrame(review_data)

# Sort by date
reviews_df = reviews_df.sort_values('Review_Date').reset_index(drop=True)

# ============================================================================
# ADD SOME MISSING VALUES (REALISTIC)
# ============================================================================

print("  → Adding realistic missing values...")

# Some reviews don't have titles
missing_titles = np.random.choice(len(reviews_df), size=int(len(reviews_df) * 0.05), replace=False)
reviews_df.loc[missing_titles, 'Review_Title'] = np.nan

# Some reviews don't track helpful votes
missing_votes = np.random.choice(len(reviews_df), size=int(len(reviews_df) * 0.10), replace=False)
reviews_df.loc[missing_votes, 'Helpful_Votes'] = np.nan

# ============================================================================
# SAVE TO CSV
# ============================================================================

print("  → Saving to reviews.csv...")
reviews_df.to_csv('reviews.csv', index=False)

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

elapsed_time = time.time() - start_time

print("\n" + "="*70)
print("✅ REVIEWS.CSV GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nTime taken: {elapsed_time:.2f} seconds")
print(f"Total reviews: {len(reviews_df):,}")
print(f"File size: {reviews_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print("\n📊 DATA SUMMARY:")
print("-"*70)
print(f"Date range: {reviews_df['Review_Date'].min()} to {reviews_df['Review_Date'].max()}")
print(f"Average rating: {reviews_df['Rating'].mean():.2f}")
print(f"Average review length: {reviews_df['Review_Length_Words'].mean():.1f} words")
print(f"Verified purchases: {reviews_df['Verified_Purchase'].sum():,} ({reviews_df['Verified_Purchase'].mean()*100:.1f}%)")

print("\n⭐ RATING DISTRIBUTION:")
print(reviews_df['Rating'].value_counts().sort_index())

print("\n😊 SENTIMENT DISTRIBUTION:")
sentiment_counts = reviews_df['Sentiment'].value_counts()
for sent, count in sentiment_counts.items():
    print(f"  {sent.capitalize()}: {count:,} ({count/len(reviews_df)*100:.1f}%)")

print("\n📝 REVIEW LENGTH DISTRIBUTION:")
length_bins = pd.cut(reviews_df['Review_Length_Words'], 
                     bins=[0, 20, 50, 100, 200], 
                     labels=['Very Short (0-20)', 'Short (20-50)', 'Medium (50-100)', 'Long (100+)'])
print(length_bins.value_counts().sort_index())

print("\n🎯 TOP ASPECTS MENTIONED:")
all_aspects = reviews_df['Aspects_Mentioned'].str.split(', ').explode()
aspect_counts = all_aspects.value_counts().head(10)
print(aspect_counts)

print("\n👤 ANONYMOUS VS NAMED REVIEWS:")
anonymous_count = (reviews_df['Reviewer_Name'] == 'Anonymous').sum()
print(f"  Anonymous: {anonymous_count:,} ({anonymous_count/len(reviews_df)*100:.1f}%)")
print(f"  Named: {len(reviews_df) - anonymous_count:,} ({(len(reviews_df) - anonymous_count)/len(reviews_df)*100:.1f}%)")

print("\n⚠️ MISSING VALUES:")
missing = reviews_df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("  No missing values")

print("\n📋 SAMPLE REVIEWS:")
print("-"*70)
sample_reviews = reviews_df.sample(3)[['Review_Title', 'Rating', 'Review_Text', 'Sentiment']]
for idx, row in sample_reviews.iterrows():
    print(f"\n{row['Review_Title']} - Rating: {row['Rating']} - Sentiment: {row['Sentiment']}")
    print(f"  {row['Review_Text'][:150]}...")

print("\n" + "="*70)
print("✨ Perfect for NLP tasks on Days 18-19!")
print("="*70)
print("\n💡 READY FOR:")
print("  ✓ Sentiment Analysis (positive/negative/neutral)")
print("  ✓ Text Classification")
print("  ✓ Aspect-Based Sentiment Analysis")
print("  ✓ Review Helpfulness Prediction")
print("  ✓ Word Embeddings (Word2Vec, GloVe)")
print("  ✓ BERT Fine-tuning")
print("  ✓ Text Preprocessing Practice")
print("="*70)

GENERATING REVIEWS.CSV FOR NLP TASKS

Generating 25,000 customer reviews...
  → Loading existing datasets...
     ✓ Loaded 10,000 customers
     ✓ Loaded 1,000 products
     ✓ Loaded 100,000 transactions
  → Setting up review templates...
  → Selecting transactions for reviews...
     ✓ 91,916 completed transactions available
     ✓ Selected 25,000 transactions for reviews
  → Generating review texts...
     Progress: 20.0% (5,000 / 25,000)
     Progress: 40.0% (10,000 / 25,000)
     Progress: 60.0% (15,000 / 25,000)
     Progress: 80.0% (20,000 / 25,000)
     Progress: 100.0% (25,000 / 25,000)
  → Assembling dataframe...
  → Adding realistic missing values...
  → Saving to reviews.csv...

✅ REVIEWS.CSV GENERATED SUCCESSFULLY!

Time taken: 28.82 seconds
Total reviews: 25,000
File size: 15.60 MB

📊 DATA SUMMARY:
----------------------------------------------------------------------
Date range: 2019-01-02 to 2025-02-20
Average rating: 4.27
Average review length: 10.8 words
Verified purch